# Robot Learning Visualizations

#### Basic Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Change some settings for better aesthetics
mpl.rcParams.update({
    "font.family": "serif",
    "mathtext.fontset": "cm",
    "axes.labelsize": 14,
    "axes.titlesize": 16,
    "legend.fontsize": 12,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "lines.linewidth": 2,
})

# Enable high-definition inline plotting
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # or 'svg' for vector


# Optional: better style
sns.set(style="darkgrid")


#### Load Evaluation Log

In [ ]:
df = pd.read_csv("eval_logs/eval_log.csv")
print("Data loaded:", df.shape)
df.head()


#### Reward per Step

In [ ]:
df.groupby("step")["reward"].sum().plot(title="Reward per Step", figsize=(10, 4))
plt.ylabel("Reward")
plt.xlabel("Step")


#### Physical Parameters over Tine (in one Plot)

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 5))
# df["x"].plot(label="x position", ax=ax)
# df["x_vel"].plot(label="x velocity", ax=ax)
# df["y_angle"].plot(label="y angle", ax=ax)
# plt.title("Selected Observations over Time")
# plt.xlabel("Step")
# plt.legend()


#### Physical Parameters over Tine (in seperate Plots)

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 7), sharex=True)

# Postitions
df["x"].plot(ax=axes[0], label=r"$x$ position")
axes[0].legend()
df["y"].plot(ax=axes[0], label=r"$y$ position")
axes[0].legend()
df["z"].plot(ax=axes[0], label=r"$z$ position")

axes[0].legend()
axes[0].set_title(r"Position over Time")
axes[0].set_ylabel(r"position $\mathrm{[m]}$")

# Angles
df["x_angle"].plot(ax=axes[1], label=r"$\theta_x$ angle")
axes[1].legend()
df["y_angle"].plot(ax=axes[1], label=r"$\theta_y$ angle")
axes[1].legend()
df["z_angle"].plot(ax=axes[1], label=r"$\theta_z$ angle")
axes[1].legend()

axes[1].set_title(r"Angle over Time")
axes[1].set_ylabel(r"Angle $\mathrm{[TODO]}$")

# Velocities
df["x_vel"].plot(ax=axes[2], label=r"$x$ velocity")
axes[1].legend()
# y_vel isnt extracted and saved yet!
# df["y_vel"].plot(ax=axes[2], label="y velocity")
# axes[1].legend()
# z_vel isnt extracted and saved yet!
# df["z_vel"].plot(ax=axes[2], label="z velocity")
# axes[1].legend()

axes[2].set_title(r"Velocity in $x$ direction over Time")
axes[2].set_ylabel(r"$\dot{x}$ $[\frac{\mathrm{m}}{\mathrm{s}}]$")
axes[2].set_xlabel("Step")


plt.tight_layout()


In [ ]:
action_cols = [col for col in df.columns if "action" in col]
df[action_cols].plot(subplots=True, figsize=(10, 6), title="Action Over Time")


In [ ]:
info_cols = [col for col in df.columns if "penalty" in col or "info_" in col]
df[info_cols].plot(subplots=True, figsize=(10, 12), title="Penalties/Infos")